In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader

import numpy as np

In [2]:
labels = np.load("labels.npy")

training_generator = DataLoader(labels, 128)

In [3]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(98304,)))
# model.add(Dense(768, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(384, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

2022-03-19 11:15:38.306458: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-19 11:15:38.315448: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-19 11:15:38.315812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-19 11:15:38.316504: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
model = tf.keras.models.load_model("production/")

OSError: No file or directory found at production/

In [4]:
model.fit(training_generator, epochs=1)

5953/5953 [==============================] - 536s 90ms/step - loss: 4.5492 - accuracy: 0.1078


In [5]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-03-19 11:24:50.578135: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-03-19 11:24:50.813501: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 201326592 exceeds 10% of free system memory.
2022-03-19 11:24:50.873773: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 201326592 exceeds 10% of free system memory.
2022-03-19 11:24:50.925749: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 201326592 exceeds 10% of free system memory.


INFO:tensorflow:Assets written to: models/model-1647703490/assets
